In [1]:
import json

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph

In [2]:
import os
os.environ['TAVILY_API_KEY'] = "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"

from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_tool = TavilySearchResults(max_results=5) 

/tmp/ipykernel_239796/573716656.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


In [4]:
from langchain_experimental.utilities import PythonREPL
from typing import Annotated

repl = PythonREPL()
@tool
def python_repl(code: Annotated[str, "The python code to execute to generate your chart."]):
    """Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Succesfully executed:\\\\n`python\\\\\\\\n{code}\\\\\\\\n`\\\\nStdout: {result}"

In [5]:


tools = [tavily_tool, python_repl]        


In [6]:
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union

from langchain.agents import create_openai_functions_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDictye

# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str 

In [15]:
import os
from dotenv import load_dotenv

load_dotenv() # will search for .env file in local folder and load variables 

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [17]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=OPENAI_API_KEY)


In [20]:
# updated

In [21]:
import logging
import os
from typing import List

from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents.agent import AgentExecutor
from langgraph.graph import StateGraph, END

In [116]:
logging.basicConfig(level=logging.INFO)

# ✅ Initialize Tavily and OpenAI Chat
llm = ChatOpenAI(model="gpt-4", temperature=0.5)
tavily_tool = TavilySearchResults(max_results=5)
tools = [tavily_tool]

In [117]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research assistant that uses web search to find information on the topic given below"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [118]:
agent = create_openai_functions_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [119]:
class ResearchState(dict):
    topic: str
    plan: str
    links: List[str]
    contents: str
    summary: str
    approved: bool

In [120]:
def research_planner(state: ResearchState):
    topic = state["topic"]
    plan = f"Research plan for '{topic}': 1) Write background 2) Research objectives"
    logging.info("Plan created.")
    return {**state, "plan": plan}

In [121]:
def information_gatherer(state: ResearchState):
    topic = state["topic"]
    logging.info(f"Gathering info on: {topic}")
    result = agent_executor.invoke({"input": f"Find recent web sources about: {topic}"})
    links = []
    if isinstance(result, dict) and "output" in result:
        output = result["output"]
        links = output.split("\n")[:3]  # naive extraction
    logging.info(f"Links gathered: {links}")
    return {**state, "links": links}

In [122]:
def content_analyzer(state: ResearchState):
    if not state.get("links"):
        raise ValueError("No links to analyze.")
    contents = "\n".join([f"Summary of {link}" for link in state["links"]])
    logging.info("Content analyzed.")
    return {**state, "contents": contents}

In [123]:

def report_generator(state: ResearchState):
    summary = f"Report on '{state['topic']}':\n{state['contents']}"
    logging.info("Report generated.")
    return {**state, "summary": summary}

In [124]:
def human_review(state: ResearchState):
    print("Do you approve this report? (yes/no)")
    approval = input()
    approved = approval.lower().strip() == "yes"
    return {**state, "approved": approved}

In [125]:

def error_recovery(state: ResearchState):
    logging.warning("Recovering from error...")
    return {**state, "links": [], "contents": "Fallback content."}

In [126]:

builder = StateGraph(ResearchState)

In [127]:
builder.add_node("Planner", RunnableLambda(research_planner))
builder.add_node("Gatherer", RunnableLambda(information_gatherer))
builder.add_node("Analyzer", RunnableLambda(content_analyzer))
builder.add_node("Reporter", RunnableLambda(report_generator))
builder.add_node("Review", RunnableLambda(human_review))
builder.add_node("Recovery", RunnableLambda(error_recovery))


In [128]:

builder.set_entry_point("Planner")

builder.add_edge("Planner", "Gatherer")
builder.add_edge("Gatherer", "Analyzer")

In [129]:
builder.add_conditional_edges(
    "Analyzer",
    lambda state: "Recovery" if not state.get("contents") else "Reporter",
    {"Recovery": "Reporter", "Reporter": "Review"}
)

builder.add_conditional_edges(
    "Review",
    lambda state: END if state.get("approved") else "Gatherer",
    {END: END, "Gatherer": "Analyzer"}
)

In [130]:
graph = builder.compile()

# ✅ Run
initial_state = ResearchState(topic="AI in modern world")
final_state = graph.invoke(initial_state)



INFO:root:Plan created.
INFO:root:Gathering info on: AI in modern world




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `tavily_search_results_json` with `{'query': 'latest advancements in artificial intelligence'}`


[{'title': 'Latest Development of Artificial Intelligence | InData Labs', 'url': 'https://indatalabs.com/blog/ai-latest-developments', 'content': '### #3: AI advancements in healthcare\n\nThe latest development of artificial intelligence in the healthcare world is enabling improvements in accurate diagnoses, clinical decision-making, and optimal patient care. [...] Some of the most recent advancements in this field include the release of Magnifier, a behavioral analytics solution from Palo Alto Networks. The latest AI developments solution models network behavior by using machine learning to enhance threat detection.\n\nAlphabet, the parent company of Google, has introduced the cybersecurity intelligence platform Chronicle to streamline and improve cybersecurity measures as we know them.\n\n### #9: AI applications in fintech [...] ## Exploring recent developments in artificial i

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Links gathered: ['Here are some of the latest advancements in the field of Artificial Intelligence:', '', '1. **AI advancements in healthcare**: AI is being used to improve accurate diagnoses, clinical decision-making, and optimal patient care. For instance, Magnifier, a behavioral analytics solution from Palo Alto Networks, models network behavior using machine learning to enhance threat detection [source](https://indatalabs.com/blog/ai-latest-developments).']
INFO:root:Content analyzed.


Here are some of the latest advancements in the field of Artificial Intelligence:

1. **AI advancements in healthcare**: AI is being used to improve accurate diagnoses, clinical decision-making, and optimal patient care. For instance, Magnifier, a behavioral analytics solution from Palo Alto Networks, models network behavior using machine learning to enhance threat detection [source](https://indatalabs.com/blog/ai-latest-developments).

2. **AI in cybersecurity**: Alphabet, the parent company of Google, introduced the cybersecurity intelligence platform Chronicle to streamline and improve cybersecurity measures [source](https://indatalabs.com/blog/ai-latest-developments).

3. **AI trends in robotics**: Many robotics startups are using AI to develop and automate new systems. For example, firms like Olis Robotics are driving innovation through the use of plug-and-play controls fitted with AI-powered software [source](https://indatalabs.com/blog/ai-latest-developments).

4. **Google's AI 

 yes


In [131]:
print("\n📄 Final Report:")
print(final_state)


📄 Final Report:
{'topic': 'AI in modern world', 'plan': "Research plan for 'AI in modern world': 1) Write background 2) Research objectives", 'links': ['Here are some of the latest advancements in the field of Artificial Intelligence:', '', '1. **AI advancements in healthcare**: AI is being used to improve accurate diagnoses, clinical decision-making, and optimal patient care. For instance, Magnifier, a behavioral analytics solution from Palo Alto Networks, models network behavior using machine learning to enhance threat detection [source](https://indatalabs.com/blog/ai-latest-developments).'], 'contents': 'Summary of Here are some of the latest advancements in the field of Artificial Intelligence:\nSummary of \nSummary of 1. **AI advancements in healthcare**: AI is being used to improve accurate diagnoses, clinical decision-making, and optimal patient care. For instance, Magnifier, a behavioral analytics solution from Palo Alto Networks, models network behavior using machine learning

In [132]:
print(final_state['topic'])
print(final_state['plan'])
print(final_state['links'])


print(final_state['contents'])


AI in modern world
Research plan for 'AI in modern world': 1) Write background 2) Research objectives
['Here are some of the latest advancements in the field of Artificial Intelligence:', '', '1. **AI advancements in healthcare**: AI is being used to improve accurate diagnoses, clinical decision-making, and optimal patient care. For instance, Magnifier, a behavioral analytics solution from Palo Alto Networks, models network behavior using machine learning to enhance threat detection [source](https://indatalabs.com/blog/ai-latest-developments).']
Summary of Here are some of the latest advancements in the field of Artificial Intelligence:
Summary of 
Summary of 1. **AI advancements in healthcare**: AI is being used to improve accurate diagnoses, clinical decision-making, and optimal patient care. For instance, Magnifier, a behavioral analytics solution from Palo Alto Networks, models network behavior using machine learning to enhance threat detection [source](https://indatalabs.com/blog/